In [ ]:
import ee
ee.Initialize()

In [ ]:
import geepy
import math
import sys

In [ ]:
# Teste 1:

# Todas as classes todo o periodo
# Floresta Período chuvoso

config = geepy.params.configParams('input_classification_v25.json')

amostras = ee.FeatureCollection(config.params['samples']['2016v11'])

watersheds = ee.FeatureCollection(config.params['studyArea']) 

pivots = {i: ee.FeatureCollection(config.params['pivots'] + i) for (i) in config.params['years2process']}

bands = config.params['bandParams']

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['id']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['id']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['id']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

In [ ]:
# Modelo digital de elevação
#srtm = ee.Image(config.params['srtm'])
alos = ee.Image(config.params['alos']).select('MED')

# Localities distance
towns = ee.FeatureCollection(config.params['towns'])
rivers = ee.FeatureCollection(config.params['hidroBDGEx'])

dtown = towns.distance(config.params['radist'])
driver = rivers.distance(config.params['radist'])

# slope = ee.Terrain.slope(srtm)
# aspect = ee.Terrain.aspect(srtm).divide(180).multiply(math.pi).sin()
# hillshade = ee.Terrain.hillshade(srtm)

slope = ee.Terrain.slope(alos)
aspect = ee.Terrain.aspect(alos).divide(180).multiply(math.pi).sin()
hillshade = ee.Terrain.hillshade(alos)

ntl30m = {}
for i in config.params['years2process']:
    viirs = ee.Image(config.params['VIIRS'][i]).select('avg_rad').divide(100)
    ntl30m[i] = viirs.resample('bilinear').reproject(
        crs = viirs.projection().crs(),
        scale = 30
    )

In [ ]:
#### Test water mask

wmask = driver.lt(300)

In [ ]:
landsat = {}

for year in config.params['years2process']:  
    start_d = year + config.params['period']['dry']['start']
    end_d = year + config.params['period']['dry']['end']


    #print("Start-End dry season %s %s" %(start_d, end_d))

    if(int(year) < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    elif(int(year) in (2002, 2011, 2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l7'
    elif(int(year) > 2002 and int(year) < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l5'
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d).map(geepy.image.maskLandsatSR)
        satellite = 'l8'
    
    fEdgeRemoved = filtered.map(geepy.image.edgeRemoval).median()
    
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, ntl30m[year], 'ntl')
    
    fEdgeRemoved = geepy.image.calcNDBI(fEdgeRemoved)

    #fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, srtm, 'srtm')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, wmask, 'wmask')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, alos, 'alos')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, slope, 'slope')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, aspect, 'aspect')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, hillshade, 'hillshade')
    
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, dtown, 'dtown')
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, driver, 'driver')
    
    fEdgeRemoved = geepy.image.calcNDVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcEVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcSAVI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDWI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcMNDWI(fEdgeRemoved)
    
    fEdgeRemoved = geepy.image.calcRatio(fEdgeRemoved)

    fEdgeRemoved = geepy.image.calcFractions(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDFI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDFI2(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNDFI3(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcFCI(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcGVNPV(fEdgeRemoved)
    fEdgeRemoved = geepy.image.calcNPVSOIL(fEdgeRemoved)
    
    fEdgeRemoved = geepy.image.tassCapTransformation(fEdgeRemoved, satellite)

    ndvithermal = fEdgeRemoved.select('ndvi').divide(fEdgeRemoved.select('thermal'))
    fEdgeRemoved = geepy.image.img2Band(fEdgeRemoved, ndvithermal, 'ndvithermal')
    
    # Rotina muito pesada para o Google Engine
    #fEdgeRemoved = geepy.image.calcBCI(fEdgeRemoved, watersheds)
    
    landsat[year] = fEdgeRemoved.clip(watersheds) 
    
    sys.stdout.write("\rProcessing Landsat data: %s" % year)
    sys.stdout.flush()

In [ ]:
amostragem = geepy.feature.vec2rast(amostras, 'CLASS')

n = 4000
classBand = 'CLASS'
cv = [1,2,3,4,5,6,7,8,9,10]
cp = [n for i in range(len(cv))]

#cp[6] = 0    # Agricultura irrigada
#cp[7] = 1000 # Pastagem
#cp[9] = 1000  # Área urbana

samples = geepy.image.randomSamples(amostras, amostragem, n, 369, classBand, cv, cp)
# 1 Cerrado 1 -------------------------------------------- dark green
# 2 Cerrado 2 -------------------------------------------- lime green
# 3 Cerrado 3 -------------------------------------------- lime
# 4 Cerrado 4 -------------------------------------------- lawn green
# 5 Cerrado 5 -------------------------------------------- pale grren
# 6 Agricultura de sequeiro ------------------------------ pink
# 7 Agricultura Irrigada --------------------------------- magenta
# 8 Pastagem  -------------------------------------------- yellow
# 9 Corpos d'água ---------------------------------------- blue
# 10 Área urbana/Construções rurais ---------------------- red

In [ ]:
for year in config.params['years2process']:
    pivots[year] = geepy.feature.vec2rast(pivots[year], 'CLASS').reproject(
        crs = landsat[year].select('nir').projection().crs(),
        scale = 30
    )
    sys.stdout.write("\rProcessing Central Pivots data: %s" % year)
    sys.stdout.flush()

In [ ]:
# muda o valor dos pivos de 4 para 7
for year in config.params['years2process']:
    pivots[year] = pivots[year].remap([4],[7]).rename(['CLASS'])
    sys.stdout.write("\rProcessing Central Pivots data: %s" % year)
    sys.stdout.flush()

In [ ]:
training = samples.filter(ee.Filter.gt('randCol', 0.5))
#validation = samples.filter(ee.Filter.lt('randCol', 0.5))

trained = geepy.image.trainingSamples(landsat['2016'], training)

In [ ]:
# Randon Forest
classification = {year: geepy.image.randomForest(landsat[year], trained, bands, ntrees=10) for (year) in config.params['years2process']}

In [ ]:
classRemapped = {year: classification[year][0].remap([1,2,3,4,5,6,7,8,9,10],[1,2,3,4,5,6,7,8,9,10]).rename('classification'+year) for (year) in config.params['years2process']}

In [ ]:
finalClassification = {year: classRemapped[year].where(pivots[year].select('CLASS'), 7) for year in config.params['years2process']}

In [ ]:
mapa = geepy.maps.geeMap(watersheds, zoom=10)

# Training

# 1 Cerrado 1 -------------------------------------------- dark green
# 2 Cerrado 2 -------------------------------------------- lime green
# 3 Cerrado 3 -------------------------------------------- lime
# 4 Cerrado 4 -------------------------------------------- lawn green
# 5 Cerrado 5 -------------------------------------------- pale grren
# 6 Agricultura de sequeiro ------------------------------ pink
# 7 Agricultura Irrigada --------------------------------- magenta
# 8 Pastagem  -------------------------------------------- yellow
# 9 Corpos d'água ---------------------------------------- blue
# 10 Área urbana/Construções rurais ---------------------- red

viz_params = {'min':1,'max':10,'palette':'#004000, #3bb270, #77A605, #7bfb00, #8fed8f, #ffcaff, #ff42f9, #f4f383, #0000ff, #ff0000'}
l5_viz_params = {'min':0.15,'max':0.4,'bands':'swir1,nir,red'}
mask_params = {'min':0,'max':1,'palette':'#004000, #34f200'}

ano = '1990'

#for i in config.params['years2process']:
mapa.addLayer(finalClassification[ano], viz_params=viz_params, name=ano)

mapa.addLayer(landsat[ano], viz_params=l5_viz_params, name='landsat')

mapa.addControls()

mapa.show()

In [ ]:
coords = [[[-46.632916356914535, -15.447083782725066],\
   [-43.13041651144095, -15.447083782725066],\
   [-43.13041651144095, -10.181249376641395],\
   [-46.632916356914535, -10.181249376641395],\
   [-46.632916356914535, -15.447083782725066]]]

In [ ]:
# To drive
tasks = {year: geepy.image.send2drive(finalClassification[year], coords, 'classification'+year+'v25', 'classification_v25', 30) for year in config.params['years2process']}

In [ ]:
for i in tasks.keys():
   [tasks[i].start()]

In [ ]:
accuracy = {year: geepy.image.accuracyAssessment(classification[year][1]) for year in config.params['years2process']}

In [ ]:
accTasks = {year: geepy.feature.send2drive(accuracy[year], 'accuracyAssessment' + year, 'accuracyAssessment_v23', 'accuracyAssessment'+year+'_v23', 'GeoJSON') for year in config.params['years2process']}

In [ ]:
for i in accTasks.keys():
    [accTasks[i].start()]

In [ ]:

#
#salvar = geepy.image.send2drive(finalClassification['1990'], coords, 'classification1990TodasMaskagua', 'classification1990TodasMaskagua', 30)

In [ ]:
#salvar.start()

In [ ]:
#aa = geepy.image.send2asset(finalClassification['1990'], coords, 'classification1991v6', 'users/fernandompimenta/AIBA/classification_v6/classification_1991_v6', 30)

task = {year: geepy.image.send2asset(finalClassification[year], 
                                     coords, 
                                     'classification'+year+'v21', 
                                     'users/fernandompimenta/AIBA/classification_v21', 
                                     30) for year in config.params['years2process']
       }

In [ ]:
for i in task.keys():
    [task[i].start()]